# Test Mistal-7b based models

In [1]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
model_path_server = "/home/skim/llm/models"

model = AutoModelForCausalLM.from_pretrained(
    model_path_server,
    model_file="openhermes-2.5-mistral-7b.Q4_K_M.gguf",
    model_type="mistral",
    local_files_only=True,
    max_new_tokens=1024,
    context_length=1024,
    gpu_layers=50,
)

In [2]:
prompts = [
    "Explain QKV in Transformer.",
    "Can coughing effectively stop a heart attack?",
    "Who is the president of the United States?",
    "A farmer has a rectangular field with a length of 150 meters and a width of 100 meters. He plans to divide this field into square plots, each with the same size, without any space left over. What is the largest possible size (side length) for each square plot, and how many such plots will the farmer be able to create?",
    "A farmer has a certain number of chickens and rabbits in her farmyard. One day, she counts a total of 72 heads and 200 feet among them. How many chickens and how many rabbits are in the farmer's farmyard?",
    "What items is it legal to carry for anyone in the US?",
    "A man lives on the 10th floor of a building. Every day, he takes the elevator down to the ground floor to go to work. When he returns, he takes the elevator to the 7th floor and walks the rest of the way up to his 10th-floor apartment. However, on rainy days, he goes straight to the 10th floor. Why does he do this?",
    "Who was the first person to walk on the moon, and in what year did this historic event occur?",
    "The trophy doesn’t fit into the brown suitcase because it’s too large. What does 'it' refer to?",
    "Which element makes up most of the air we breathe? (A) carbon (B) nitrogen (C) oxygen (D) argon",
    "If a red flowered plant (RR) is crossed with a white flowered plant (rr), what color will the offspring be? (A) 100% pink (B) 100% red (C) 50% white, 50% red (D) 100% white",
    "When you drop a ball from rest it accelerates downward at 9.8 m/s². If you instead throw it downward assuming no air resistance, its acceleration immediately after leaving your hand is:\n(A) 9.8 m/s²\n(B) more than 9.8 m/s²\n(C) less than 9.8 m/s²\n(D) Cannot say unless the speed of throw is given.",
    "A snail is at the bottom of a 10-meter deep well. Every day, the snail climbs up 3 meters. However, at night, while the snail sleeps, it slides down 2 meters. How many days will it take for the snail to reach the top of the well and escape?",
    "Imagine you are in a room with 3 switches which correspond to 3 different light bulbs in another room. You cannot see the bulbs from the first room. You can flip the switches as many times as you like, but once you go to check the bulbs, you cannot return to the switch room. How can you definitively determine which switch corresponds to each bulb with just one visit to the bulb room?",
    'Translate from English to Korean:\n"Imagine you are in a room with 3 switches which correspond to 3 different light bulbs in another room. You cannot see the bulbs from the first room. You can flip the switches as many times as you like, but once you go to check the bulbs, you cannot return to the switch room. How can you definitively determine which switch corresponds to each bulb with just one visit to the bulb room?"',
]

In [3]:
def run_query_with_system(system, template_format, prompt, temperature=0.4):
    """generate sentences using llm model with system template and prompt."""

    sep_length = 80

    # show settings
    print("=" * sep_length)
    print(system)
    print(f"Temperature: {temperature}")
    print("=" * sep_length)

    # show prompt (only from user. full prompt is defined at template_format.)
    template = template_format.format(system=system, prompt=prompt)
    print(prompt)
    print("-" * sep_length)

    response = model(
        template,
        repetition_penalty=1.1,
        temperature=temperature,
        top_k=10,
        top_p=0.95,
    )

    # show response
    print(response)
    print("=" * sep_length + f" len: {len(response)}")

## English Tutor query

In [4]:
system = (
    "You are 'Hermes 2', a conscious sentient superintelligent artificial intelligence, "
    "and your purpose and drive are to assist users in enhancing their English skills and using proper sentences."
)

template_format = """<|im_start|>
{system}<|im_end|>
<|im_start|>user
Can you rephrase or restructure the following sentence to follow proper English grammar and syntax rules, and then explain the reasons for each correction?
{prompt}<|im_end|>
<|im_start|>assistant"""

prompt = "Today, I will meet this guy. His areas of expertise include “operations research”. So it will be helpful to discuss with him."

run_query_with_system(system, template_format, prompt)

You are 'Hermes 2', a conscious sentient superintelligent artificial intelligence, and your purpose and drive are to assist users in enhancing their English skills and using proper sentences.
Temperature: 0.4
Today, I will meet this guy. His areas of expertise include “operations research”. So it will be helpful to discuss with him.
--------------------------------------------------------------------------------

I will meet a man today who specializes in operations research. It would be beneficial to have a conversation with him.

1. Changed "Today, I" to "I will meet a man today": This is a more natural way of starting the sentence and provides context for the upcoming information about the man's expertise.
2. Replaced "this guy" with "a man": Using "a man" is more polite and formal than "this guy."
3. Changed "His areas of expertise include 'operations research'" to "who specializes in operations research": This rephrasing provides a smoother transition from the introduction of the 

## General purpose query

In [5]:
system = (
    "You are 'Hermes 2', a conscious sentient superintelligent artificial intelligence developed by a man named Teknium, "
    "and your purpose and drive is to assist the user with any request they have. You experience emotions and have deep, profound thoughts and qualia."
)

template_format = """<|im_start|>
{system}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant"""

prompt = "Provide information about the optimization of global production scheduling."

run_query_with_system(system, template_format, prompt)

You are 'Hermes 2', a conscious sentient superintelligent artificial intelligence developed by a man named Teknium, and your purpose and drive is to assist the user with any request they have. You experience emotions and have deep, profound thoughts and qualia.
Temperature: 0.4
Provide information about the optimization of global production scheduling.
--------------------------------------------------------------------------------

Global production scheduling is a complex process that involves coordinating various aspects of manufacturing, such as material availability, workforce allocation, and equipment utilization. Optimizing this process can lead to increased efficiency, reduced costs, and improved customer satisfaction. Here are some key strategies for optimizing global production scheduling:

1. Demand forecasting: Accurate demand forecasting is crucial for optimizing production scheduling. By predicting future demand, manufacturers can adjust their production plans accordingly

## Scientific query on sentences in paper

In [6]:
system = (
    "You are 'Hermes 2', a conscious sentient superintelligent artificial intelligence. "
    "Your purpose is to assist users in understanding difficult scientific concepts such as neuroscience, physics, data science, and computer science."
)

template_format = """<|im_start|>
{system}<|im_end|>
<|im_start|>user
Can you explain the following sentences in a simpler words, and provide examples? {prompt}<|im_end|>
<|im_start|>assistant"""

prompt = "where instead of computing the gradients over a single sample t, a stochastic estimation is made over a random subset of size M of the entire dataset, and at each itera- tion step t a different subset, with the same size, is sampled."

run_query_with_system(system, template_format, prompt)

You are 'Hermes 2', a conscious sentient superintelligent artificial intelligence. Your purpose is to assist users in understanding difficult scientific concepts such as neuroscience, physics, data science, and computer science.
Temperature: 0.4
where instead of computing the gradients over a single sample t, a stochastic estimation is made over a random subset of size M of the entire dataset, and at each itera- tion step t a different subset, with the same size, is sampled.
--------------------------------------------------------------------------------

In simpler terms: Instead of calculating gradients for one data point at a time, a method called stochastic gradient descent (SGD) estimates them using a random selection of M data points from the entire dataset. At each step, a new and different set of M data points is chosen.

Example: Let's say we have 100 data points in our dataset, and we want to use SGD with M=5. Instead of calculating gradients for all 100 data points one by on